In [1]:
from pathlib import Path as pt
import pandas as pd
import json
import numpy as np
import sigfig
from uncertainties import ufloat, ufloat_fromstr

# ufloat_fromstr('1.24(4)') > ufloat_fromstr('1.23(3)')

In [2]:
from load_data import (
    processed_data_dirs, plots_dir, titles, total_counts,
    models, models_labels, embeddings_dirname, embeddings_names, root_loc
)

print(
    # plots_dir.parent, 
    # [d.name for d in processed_data_dirs], 
    # plots_dir.name, 
    # titles, 
    # total_counts, 
    # models, 
    # models_labels, 
    # embeddings_dirname, 
    embeddings_names, 
    root_loc
)

['Mol2Vec', 'VICGAE'] /Users/aravindhnivas/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/ML-properties


In [4]:
def df_to_latex(
    df, caption='<caption here>', label='tab:<label here>', column_format=None, float_format='{:.2f}', 
):
    """
    Convert pandas DataFrame to LaTeX table string.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to convert
    caption : str, optional
        Table caption
    label : str, optional
        Table label for referencing
    column_format : str, optional
        LaTeX column format (e.g., 'lcc' for left-aligned, centered, centered)
        If None, will generate 'l' for string columns and 'c' for numeric columns
    float_format : str, optional
        Format string for floating point numbers
        
    Returns:
    --------
    str : LaTeX table code
    """
    # Deep copy to avoid modifying original DataFrame
    df = df.copy()
    
    # Generate column format if not provided
    if column_format is None:
        column_format = ''
        for col in df.columns:
            if df[col].dtype in ['float64', 'int64']:
                column_format += 'c'
            else:
                column_format += 'l'
    
    # Format floating point numbers
    for col in df.select_dtypes(include=['float64']):
        df[col] = df[col].apply(lambda x: float_format.format(x))
    
    # Start building LaTeX table
    latex = ['\\begin{table}[!htb]']
    latex.append('\t\\centering')
    
    # Add caption if provided
    if caption:
        latex.append(f'\t\\caption{{{caption}}}')
    
    # Add label if provided
    if label:
        latex.append(f'\t\\label{{{label}}}')
    
    # Begin tabular environment
    latex.append(f'\t\\begin{{tabular}}{{{column_format}}}')
    latex.append('\t\t\\toprule')
    
    # Add header
    headers = ' & '.join(df.columns)
    latex.append(f'\t\t{headers} \\\\')
    latex.append('\t\t\\midrule')
    
    # Add rows
    for _, row in df.iterrows():
        latex.append('\t\t' + ' & '.join(str(val) for val in row) + ' \\\\')
    
    # End tabular environment
    latex.append('\t\t\\bottomrule')
    latex.append('\t\\end{tabular}')
    latex.append('\\end{table}')
    
    return '\n'.join(latex)

### Data shape pre- and post-processing

In [5]:
metadata = {}
rows_data_shapes = []

for idx, dirs in enumerate(processed_data_dirs):
    title = titles[idx]
    metadata[title] = {}
    
    embedded_vectors_dir = dirs / 'embedded_vectors'
    for embedder, embedding in zip(embeddings_names, embeddings_dirname):
        metadata[title][embedder] = {}
        metadata_file = embedded_vectors_dir / f'processed_{embedding}' / 'metadata.json'
        with open(metadata_file, 'r') as f:
            meta = json.load(f)
            metadata[title][embedder] = meta
            original = meta['original_X_shape']
            validated = meta['validated_X_shape']
            cleaned = meta['cleaned_X_shape']
            rows_data_shapes.append([title, embedder, original, validated, cleaned])
        
# print(json.dumps(metadata, indent=4))
data_shapes_df = pd.DataFrame(rows_data_shapes, columns=['Property', 'Embedder', 'Original', 'Validated', 'Cleaned'])
data_shapes_df.to_csv(root_loc / 'results/data_shapes.csv', index=False)
data_shapes_df

,Property,Embedder,Original,Validated,Cleaned
0,MP,Mol2Vec,7476,7476,5980
1,MP,VICGAE,7476,7200,5940
2,BP,Mol2Vec,4915,4915,4865
3,BP,VICGAE,4915,4909,4295
4,VP,Mol2Vec,398,398,348
5,VP,VICGAE,398,398,338
6,CP,Mol2Vec,777,777,769
7,CP,VICGAE,777,776,727
8,CT,Mol2Vec,819,819,802
9,CT,VICGAE,819,818,818


In [48]:
latex_str = df_to_latex(data_shapes_df)
latex_str = latex_str.replace('\t', '    ')
print(latex_str)

\begin{table}[!htb]
    \centering
    \caption{<caption here>}
    \label{tab:<label here>}
    \begin{tabular}{llccc}
        \toprule
        Property & Embedder & Original & Validated & Cleaned \\
        \midrule
        MP & Mol2Vec & 7476 & 7476 & 5980 \\
        MP & VICGAE & 7476 & 7200 & 5940 \\
        BP & Mol2Vec & 4915 & 4915 & 4865 \\
        BP & VICGAE & 4915 & 4909 & 4295 \\
        VP & Mol2Vec & 398 & 398 & 348 \\
        VP & VICGAE & 398 & 398 & 338 \\
        CP & Mol2Vec & 777 & 777 & 769 \\
        CP & VICGAE & 777 & 776 & 727 \\
        CT & Mol2Vec & 819 & 819 & 802 \\
        CT & VICGAE & 819 & 818 & 818 \\
        \bottomrule
    \end{tabular}
\end{table}


In [6]:
modes = ['default', 'default_cleaned_xgboost', 'best_model', 'best_model_cleaned_xgboost']
cleaned_model = 'xgboost'

nfold = 5
metrics_rows = []

invalid_entries = 0

for dir_ind, dir in enumerate(processed_data_dirs):
    for model_ind, model in enumerate(models):
        for embedder_ind, embedder in enumerate(embeddings_dirname):
            for mode in modes:
                property_name = titles[dir_ind]
                model_name = models_labels[model_ind]
                embedder_name = embeddings_names[embedder_ind]
                print(f"Processing {property_name}: {model_name} - {embedder_name}")
                
                pre_trained_filename = f'{model}_{embedder}_pretrained_model_{mode}'
                loc = dir / f'pretrained_models/{model}/{embedder}/{mode}/'

                cv_scores_file = loc / f'{pre_trained_filename}.cv_scores.json'
                if not cv_scores_file.exists():
                    print(f"Skipping {cv_scores_file.name}\n")
                    metrics_row = [property_name, model_name, embedder_name, mode, np.nan, np.nan, np.nan]
                    metrics_rows.append(metrics_row)
                    invalid_entries += 1
                    continue
                
                cv_scores_info = json.load(open(cv_scores_file, 'r'))
                cv_scores = cv_scores_info[f'{nfold}']
                
                # get the mean and std of the test scores
                test_scores = cv_scores['test']
                r2 = sigfig.round(test_scores['r2']['mean'], test_scores['r2']['std'], sep="external_brackets")
                rmse = sigfig.round(test_scores['rmse']['mean'], test_scores['rmse']['std'], sep="external_brackets")
                mae = sigfig.round(test_scores['mae']['mean'], test_scores['mae']['std'], sep="external_brackets")
                
                print(f"R2: {r2}, RMSE: {rmse}, MAE: {mae}")
                metrics_row = [property_name, model_name, embedder_name, mode, r2, rmse, mae]
                metrics_rows.append(metrics_row)
                print()
            print()
        print()
        
    print("#".center(80, '#') + '\n')

print(f"Invalid entries: {invalid_entries}")

Processing MP: GBR - Mol2Vec
R2: 0.67(3), RMSE: 59(3), MAE: 45(2)

Processing MP: GBR - Mol2Vec
R2: 0.83(1), RMSE: 39(1), MAE: 31(1)

Processing MP: GBR - Mol2Vec
R2: 0.71(4), RMSE: 54(4), MAE: 40(2)

Processing MP: GBR - Mol2Vec
R2: 0.87(1), RMSE: 35(2), MAE: 27(1)


Processing MP: GBR - VICGAE
R2: 0.56(3), RMSE: 67(2), MAE: 52(1)

Processing MP: GBR - VICGAE
R2: 0.72(1), RMSE: 49(2), MAE: 39(1)

Processing MP: GBR - VICGAE
R2: 0.63(3), RMSE: 61(3), MAE: 47(2)

Processing MP: GBR - VICGAE
R2: 0.80(1), RMSE: 42(2), MAE: 33.1(9)



Processing MP: CatBoost - Mol2Vec
R2: 0.71(4), RMSE: 54(4), MAE: 40(2)

Processing MP: CatBoost - Mol2Vec
R2: 0.83(1), RMSE: 40(2), MAE: 31(1)

Processing MP: CatBoost - Mol2Vec
R2: 0.69(4), RMSE: 56(4), MAE: 42(2)

Processing MP: CatBoost - Mol2Vec
R2: 0.854(9), RMSE: 36(1), MAE: 28(1)


Processing MP: CatBoost - VICGAE
R2: 0.64(3), RMSE: 61(2), MAE: 46(1)

Processing MP: CatBoost - VICGAE
R2: 0.80(1), RMSE: 42(1), MAE: 33.1(7)

Processing MP: CatBoost - VIC

In [11]:
8 * 5 * 4

160

In [95]:
metrics_columns = ["Property", "Model", "Embedder", "Mode", "R2", "RMSE", "MAE"]
metrics_rows
metrics_df = pd.DataFrame(metrics_rows, columns=metrics_columns)
metrics_df.to_csv(root_loc / 'results/metrics.csv', index=False)
metrics_df

,Property,Model,Embedder,Mode,R2,RMSE,MAE
0,MP,GBR,Mol2Vec,default,0.67(3),59(3),45(2)
1,MP,GBR,Mol2Vec,default_cleaned_xgboost,0.83(1),39(1),31(1)
2,MP,GBR,Mol2Vec,best_model,0.71(4),54(4),40(2)
3,MP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.87(1),35(2),27(1)
4,MP,GBR,VICGAE,default,0.56(3),67(2),52(1)
...,...,...,...,...,...,...,...
155,CT,LGBM,Mol2Vec,best_model_cleaned_xgboost,0.93(1),32(4),23(3)
156,CT,LGBM,VICGAE,default,0.86(2),45(3),33(3)
157,CT,LGBM,VICGAE,default_cleaned_xgboost,0.86(2),45(3),33(3)
158,CT,LGBM,VICGAE,best_model,0.86(1),46(2),34(2)


In [80]:
def get_best_metrics(df: pd.DataFrame, search_width = 8, optimize='mae', savename="", save=False, verbose=False):
    if optimize not in ['r2', 'rmse', 'mae']:
        raise ValueError("optimize must be one of 'r2', 'rmse', 'mae'")
    if verbose:
        print(f'Dataframe shape: {df.shape} with columns: {df.columns}. Searching for best metrics in {search_width} rows.')
        print(f"Optimizing for {optimize}. {len(df)=}")
        
    best_metrics_rows = []
    for mrow in range(0, len(df), search_width):
        if verbose:
            print(f"Searching in rows {mrow} to {mrow+search_width}")
        r = df.iloc[mrow:mrow+search_width].to_numpy()
        if verbose:
            print(f'Current rows:\n{r}')
        best_r2 = ufloat_fromstr(r[0][4])
        best_rmse = ufloat_fromstr(r[0][5])
        best_mae = ufloat_fromstr(r[0][6])
        
        best_row = r[0]

        for _ in r[1:]:
            if optimize == 'r2':
                r2 = ufloat_fromstr(_[4])
                if r2 > best_r2:
                    best_r2 = r2
                    best_row = _
            elif optimize == 'rmse':
                rmse = ufloat_fromstr(_[5])
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_row = _
            elif optimize == 'mae':
                mae = ufloat_fromstr(_[6])
                if mae < best_mae:
                    best_mae = mae
                    best_row = _
        if verbose:
            print('Best row:', best_row)
            print()
        best_metrics_rows.append(best_row)
    
    best_metrics_df = pd.DataFrame(best_metrics_rows, columns=metrics_columns)
    if save:
        best_metrics_df.to_csv(root_loc / f'results/best_metrics{savename}_{optimize}.csv', index=False)
        print("Saved best metrics to csv file.")
        
    return best_metrics_df

In [96]:
save = True
save = False
verbose = True
verbose = False
best_metrics_df = get_best_metrics(metrics_df, search_width=4, savename='', save=save, verbose=verbose)
best_metrics_df_overall = get_best_metrics(metrics_df, search_width=8, savename='_overall', save=save)
best_metrics_df.head(10)

,Property,Model,Embedder,Mode,R2,RMSE,MAE
0,MP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.87(1),35(2),27(1)
1,MP,GBR,VICGAE,best_model_cleaned_xgboost,0.80(1),42(2),33.1(9)
2,MP,CatBoost,Mol2Vec,best_model_cleaned_xgboost,0.854(9),36(1),28(1)
3,MP,CatBoost,VICGAE,default_cleaned_xgboost,0.80(1),42(1),33.1(7)
4,MP,XGBoost,Mol2Vec,best_model_cleaned_xgboost,0.83(1),39(2),30(2)
5,MP,XGBoost,VICGAE,default_cleaned_xgboost,0.77(2),45(2),36(1)
6,MP,LGBM,Mol2Vec,best_model_cleaned_xgboost,0.87(1),35(1),27(1)
7,MP,LGBM,VICGAE,best_model_cleaned_xgboost,0.80(1),42(2),33(1)
8,BP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.91(1),27(2),18(2)
9,BP,GBR,VICGAE,best_model_cleaned_xgboost,0.915(5),25(1),19(1)


In [97]:
best_metrics_df_overall.head(10)

,Property,Model,Embedder,Mode,R2,RMSE,MAE
0,MP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.87(1),35(2),27(1)
1,MP,CatBoost,Mol2Vec,best_model_cleaned_xgboost,0.854(9),36(1),28(1)
2,MP,XGBoost,Mol2Vec,best_model_cleaned_xgboost,0.83(1),39(2),30(2)
3,MP,LGBM,Mol2Vec,best_model_cleaned_xgboost,0.87(1),35(1),27(1)
4,BP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.91(1),27(2),18(2)
5,BP,CatBoost,Mol2Vec,default_cleaned_xgboost,0.911(8),26(2),18(1)
6,BP,XGBoost,VICGAE,default_cleaned_xgboost,0.899(9),27.0(6),20.8(5)
7,BP,LGBM,Mol2Vec,default_cleaned_xgboost,0.90(1),28(2),20(2)
8,VP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.3(2),2.3(3),1.7(2)
9,VP,CatBoost,Mol2Vec,default,0.33(4),1.9(1),1.46(7)


In [99]:
best_metrics_df_overall_model = get_best_metrics(
    best_metrics_df_overall, 
    search_width=4, 
    optimize='mae', 
    savename='_overall_model', 
    # save=True, 
    # verbose=True
)
best_metrics_df_overall_model

,Property,Model,Embedder,Mode,R2,RMSE,MAE
0,MP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.87(1),35(2),27(1)
1,BP,GBR,Mol2Vec,best_model_cleaned_xgboost,0.91(1),27(2),18(2)
2,VP,CatBoost,Mol2Vec,default,0.33(4),1.9(1),1.46(7)
3,CP,CatBoost,Mol2Vec,default_cleaned_xgboost,0.91(1),0.35(4),0.21(2)
4,CT,CatBoost,Mol2Vec,default_cleaned_xgboost,0.95(1),28(5),19(3)


In [107]:
i = 1
latex_str = ''
nmax = 8
property_names = ['MP', 'BP', 'VP', 'CP', 'CT']
property_ind = 0

highlight = True
highlight = False

multirow = True
multirow = False

for br in best_metrics_df_overall_model.iterrows():
    br = br[1].to_list()
    
    if multirow:
        if i == 1:
            latex_str += f"\\midrule\n\\multirow{{{nmax}}}{{*}}{{{property_names[property_ind]}}}\n"
            property_ind += 1
    else:
        latex_str += property_names[property_ind]
        property_ind += 1
    
    # if i == 1:
    
        
    if highlight and br[2] == 'Mol2Vec':
        latex_str += f" & {br[1]} & \\textbf{{{br[2]}}} & \\textbf{{{br[4]}}} & \\textbf{{{br[5]}}} & \\textbf{{{br[6]}}} \\\\\n"
    else:
        latex_str += f" & {br[1]} & {br[2]} & {br[4]} & {br[5]} & {br[6]} \\\\\n"
        
    if i > 7:
        i = 1
        continue
    i += 1

latex_str += '\\bottomrule\n'
print(latex_str)

MP & GBR & Mol2Vec & 0.87(1) & 35(2) & 27(1) \\
BP & GBR & Mol2Vec & 0.91(1) & 27(2) & 18(2) \\
VP & CatBoost & Mol2Vec & 0.33(4) & 1.9(1) & 1.46(7) \\
CP & CatBoost & Mol2Vec & 0.91(1) & 0.35(4) & 0.21(2) \\
CT & CatBoost & Mol2Vec & 0.95(1) & 28(5) & 19(3) \\
\bottomrule



In [15]:
# best_metrics_df
# get all rows with the Embedder as Mol2Vec
mol2vec_rows = best_metrics_df[best_metrics_df['Embedder'] == 'Mol2Vec']
mol2vec_rows
metrics = {}
for i, row in mol2vec_rows.iterrows():
    # print(row['Property'], row['Model'], row['R2'], row['RMSE'], row['MAE'])
    if row['Property'] not in metrics:
        metrics[row['Property']] = {
            'R2': [row['R2']],
            'RMSE': [row['RMSE']],
            'MAE': [row['MAE']]
        }
    else:
        metrics[row['Property']]['R2'].append(row['R2'])
        metrics[row['Property']]['RMSE'].append(row['RMSE'])
        metrics[row['Property']]['MAE'].append(row['MAE'])

print(metrics)

{'MP': {'R2': ['0.87(1)', '0.854(9)', '0.83(1)', '0.87(1)'], 'RMSE': ['35(2)', '36(1)', '39(2)', '35(1)'], 'MAE': ['27(1)', '28(1)', '30(2)', '27(1)']}, 'BP': {'R2': ['0.91(1)', '0.89(1)', '0.86(1)', '0.90(1)'], 'RMSE': ['27(2)', '29(3)', '33(3)', '29(3)'], 'MAE': ['18(2)', '20(2)', '24(2)', '20(2)']}, 'VP': {'R2': ['0.3(2)', '0.3(2)', '0.2(2)', '0.3(2)'], 'RMSE': ['2.3(3)', '2.4(2)', '2.5(2)', '2.4(4)'], 'MAE': ['1.7(2)', '1.8(2)', '1.9(1)', '1.8(3)']}, 'CP': {'R2': ['0.91(1)', '0.88(3)', '0.88(2)', '0.88(2)'], 'RMSE': ['0.34(3)', '0.40(4)', '0.40(4)', '0.39(4)'], 'MAE': ['0.22(2)', '0.27(3)', '0.27(3)', '0.27(3)']}, 'CT': {'R2': ['0.94(1)', '0.92(2)', '0.910(9)', '0.93(1)'], 'RMSE': ['28(4)', '33(5)', '36(2)', '32(4)'], 'MAE': ['20(3)', '24(4)', '26(1)', '23(3)']}}
